# Packaging Open Datasets

Anyone that has worked with datasets has, at some point, wished for a package manager for datasets. A "Git for Data". Many projects have tried and many have died in the process. 

This notebook walks over a simple and effective way we could create Datasets Packages using the [Frictionless Data](https://frictionlessdata.io/) library and specs.

In [1]:
%pip install frictionless --quiet


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from frictionless import Package, Resource
import duckdb

import pandas as pd

In [3]:
duckdb.sql(
    """
    INSTALL httpfs;
    LOAD httpfs;
    """
)

## Frictionless Data Packages

Frictionless Data Packages can be simplified to a JSON/YAML file with some metadata on it. Let's inspect an exising package, the [CO2 PPM - Trends in Atmospheric Carbon Dioxide](https://datahub.io/core/co2-ppm) Package. 


In [4]:
from frictionless import Package, Resource
import duckdb

In [5]:
co2_package = Package("https://datahub.io/core/co2-ppm/datapackage.json")

In [ ]:
print(co2_package.to_yaml())

Packages have Resources, which are the tables and files.

In [7]:
co2_resource = co2_package.get_table_resource("co2-mm-mlo")

Since these resources are just files, we can work with them as we would with any other file. 
In this case, we'll use DuckDB to run SQL queries on the data.

In [8]:
duckdb.sql(f"select * from '{co2_resource.path}' order by Date desc limit 5")

┌─────────┬──────────────┬─────────┬──────────────┬────────┬────────────────┐
│  Date   │ Decimal Date │ Average │ Interpolated │ Trend  │ Number of Days │
│ varchar │    double    │ double  │    double    │ double │     int64      │
├─────────┼──────────────┼─────────┼──────────────┼────────┼────────────────┤
│ 2018-09 │     2018.708 │  405.51 │       405.51 │ 409.02 │             29 │
│ 2018-08 │     2018.625 │  406.99 │       406.99 │  408.9 │             30 │
│ 2018-07 │     2018.542 │  408.71 │       408.71 │ 408.32 │             27 │
│ 2018-06 │     2018.458 │  410.79 │       410.79 │ 408.49 │             29 │
│ 2018-05 │     2018.375 │  411.24 │       411.24 │ 407.91 │             24 │
└─────────┴──────────────┴─────────┴──────────────┴────────┴────────────────┘

## Packaging External Datasets

Now, lets try packaging an existing dataset using the Frictionless Specs. In this case, the Global Monitoring Laboratory CO2 trend. 

They're maintaining the CSV. We're going to just package it so people can benefit from the Frictionless ecosystem.

To do that, we create a `Resource` and then the `Package` with the `Resource` as a resource.

In [9]:
external_resource = Resource(
    "https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.csv"
)

The simplest possible Package:

In [10]:
external_package = Package(
    name="co2-mm-mlo",
    title="Trends in Atmospheric Carbon Dioxide",
    resources=[external_resource],
)

In [11]:
duckdb.sql(f"select * from '{external_resource.path}' order by year desc limit 5")

┌───────┬───────┬──────────────┬─────────┬────────────────┬───────┬────────┬────────┐
│ year  │ month │ decimal date │ average │ deseasonalized │ ndays │  sdev  │  unc   │
│ int64 │ int64 │    double    │ double  │     double     │ int64 │ double │ double │
├───────┼───────┼──────────────┼─────────┼────────────────┼───────┼────────┼────────┤
│  2023 │     1 │    2023.0417 │  419.47 │         419.14 │    31 │    0.4 │   0.14 │
│  2023 │     2 │     2023.125 │  420.41 │         419.49 │    25 │   0.64 │   0.25 │
│  2022 │     1 │    2022.0417 │  418.19 │         417.86 │    29 │   0.73 │   0.26 │
│  2022 │     2 │     2022.125 │  419.28 │         418.36 │    27 │   0.92 │   0.34 │
│  2022 │     3 │    2022.2083 │  418.81 │         417.32 │    30 │   0.78 │   0.27 │
└───────┴───────┴──────────────┴─────────┴────────────────┴───────┴────────┴────────┘

Now, for this to become an actual package it needs to be published somewhere. Data Packages are just URLs with a `datapackage.json/yaml` file in them.

In [12]:
print(external_package.to_yaml("/tmp/datapackage.yaml"))

$frictionless: package/v2
name: co2-mm-mlo
title: Trends in Atmospheric Carbon Dioxide
resources:
  - name: co2_mm_mlo
    type: table
    path: https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.csv
    scheme: https
    format: csv
    mediatype: text/csv



Let's upload it to a random temporal hosting service, and then use the package as other users would use it.

In [13]:
%%bash --out temp_file_path
curl --upload-file /tmp/datapackage.yaml https://transfer.sh/datapackage.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   306  100    43  100   263     24    149  0:00:01  0:00:01 --:--:--   173


The package is now published at the `temp_file_path` URL.

In [14]:
temp_file_path

'https://transfer.sh/DcgYPi/datapackage.yaml'

In [15]:
remote_external_package = Package(temp_file_path)

In [16]:
duckdb.sql(
    f"select * from '{remote_external_package.get_resource('co2_mm_mlo').path}' order by year desc limit 5"
)

┌───────┬───────┬──────────────┬─────────┬────────────────┬───────┬────────┬────────┐
│ year  │ month │ decimal date │ average │ deseasonalized │ ndays │  sdev  │  unc   │
│ int64 │ int64 │    double    │ double  │     double     │ int64 │ double │ double │
├───────┼───────┼──────────────┼─────────┼────────────────┼───────┼────────┼────────┤
│  2023 │     1 │    2023.0417 │  419.47 │         419.14 │    31 │    0.4 │   0.14 │
│  2023 │     2 │     2023.125 │  420.41 │         419.49 │    25 │   0.64 │   0.25 │
│  2022 │     1 │    2022.0417 │  418.19 │         417.86 │    29 │   0.73 │   0.26 │
│  2022 │     2 │     2022.125 │  419.28 │         418.36 │    27 │   0.92 │   0.34 │
│  2022 │     3 │    2022.2083 │  418.81 │         417.32 │    30 │   0.78 │   0.27 │
└───────┴───────┴──────────────┴─────────┴────────────────┴───────┴────────┴────────┘

Now we know how to package an external dataset. Wouldn't it be cool if we could create some sort of collections? Enter `Catalog`s.

## Create a Catalog

[A Catalog](https://framework.frictionlessdata.io/docs/framework/catalog.html) is a collection of Data Packages. It's a way to group them together.

In [17]:
from frictionless import Catalog, Dataset

In [18]:
d1 = Dataset(name="co2-mm-mlo", package=co2_package)
d2 = Dataset(name="external-co2-mm-mlo", package=remote_external_package)
catalog = Catalog(datasets=[d1, d2])

In [19]:
print(catalog.to_yaml())

datasets:
  - name: co2-mm-mlo
    package: https://datahub.io/core/co2-ppm/datapackage.json
  - name: external-co2-mm-mlo
    package: https://transfer.sh/DcgYPi/datapackage.yaml



One could imagine lots of tooling around this, a simple one would be a way for a database to understand the Catalog and allow SQL queries to be run on the packages.

Let's try to create a simple DuckDB Catalog.

In [20]:
for c in catalog.datasets:
    for resource in c.package.resources:
        if resource.type == "table" and resource.format == "csv":
            duckdb.sql(
                f"""
            create view if not exists {resource.name.replace('-', '_')} as 
            select * from '{resource.path}';
            """
            )
            print(f"Added {resource.name} table.")

Added co2-mm-mlo_csv table.
Added co2-annmean-mlo_csv table.
Added co2-gr-mlo_csv table.
Added co2-mm-gl_csv table.
Added co2-annmean-gl_csv table.
Added co2-gr-gl_csv table.
Added co2-mm-mlo table.
Added co2-annmean-mlo table.
Added co2-gr-mlo table.
Added co2-mm-gl table.
Added co2-annmean-gl table.
Added co2-gr-gl table.
Added co2_mm_mlo table.


In [21]:
duckdb.sql(
    """
SHOW TABLES;
"""
)

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ co2_annmean_gl      │
│ co2_annmean_gl_csv  │
│ co2_annmean_mlo     │
│ co2_annmean_mlo_csv │
│ co2_gr_gl           │
│ co2_gr_gl_csv       │
│ co2_gr_mlo          │
│ co2_gr_mlo_csv      │
│ co2_mm_gl           │
│ co2_mm_gl_csv       │
│ co2_mm_mlo          │
│ co2_mm_mlo_csv      │
├─────────────────────┤
│       12 rows       │
└─────────────────────┘

Now, all the datasets are a SQL away. Data is only readed when needed and this could even work in a WASM environment (using the [JSON extension](https://duckdb.org/docs/extensions/json.html))!

In [22]:
duckdb.sql(
    """
with base as (
    select 
        *, 
    date_part('year', strptime(Date, '%Y-%m')) as year 
    from co2_mm_gl order by Date desc limit 5
)

select * from co2_gr_mlo as co2
left join base on co2.year = base.year - 3
where Date is not null
order by co2.Year 
desc limit 5;
"""
)

┌───────┬─────────────────┬─────────────┬─────────┬──────────────┬─────────┬────────┬───────┐
│ Year  │ Annual Increase │ Uncertainty │  Date   │ Decimal Date │ Average │ Trend  │ year  │
│ int64 │     double      │   double    │ varchar │    double    │ double  │ double │ int64 │
├───────┼─────────────────┼─────────────┼─────────┼──────────────┼─────────┼────────┼───────┤
│  2015 │            3.02 │        0.11 │ 2018-03 │     2018.208 │  408.57 │ 406.92 │  2018 │
│  2015 │            3.02 │        0.11 │ 2018-04 │     2018.292 │  408.88 │ 407.06 │  2018 │
│  2015 │            3.02 │        0.11 │ 2018-05 │     2018.375 │  408.75 │ 407.14 │  2018 │
│  2015 │            3.02 │        0.11 │ 2018-06 │     2018.458 │  407.86 │ 407.34 │  2018 │
│  2015 │            3.02 │        0.11 │ 2018-07 │     2018.542 │  406.39 │ 407.74 │  2018 │
└───────┴─────────────────┴─────────────┴─────────┴──────────────┴─────────┴────────┴───────┘

### Persisting the Catalog

- Write the Catalog to a YAML file, put the file on IPFS
- Do the same for all the resources
- Monkeypatch Frictionless classes to be able to read IPFS URLs (fsspec)
- Add a Metadata field to the Catalog with the previous hash of the file if the file changes, the hash changes and the Catalog is updated